# 抓取台股所有股票的6月營收

In [1]:
import pandas as pd
import requests
from io import StringIO
import time
def monthly_report(year, month):
    
    # 假如是西元，轉成民國
    if year > 1990:
        year -= 1911
    
    url = 'https://mops.twse.com.tw/nas/t21/sii/t21sc03_'+str(year)+'_'+str(month)+'_0.html'
    if year <= 98:
        url = 'https://mops.twse.com.tw/nas/t21/sii/t21sc03_'+str(year)+'_'+str(month)+'.html'
    
    # 偽瀏覽器
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    
    # 下載該年月的網站，並用pandas轉換成 dataframe
    r = requests.get(url, headers=headers)
    r.encoding = 'big5'

    dfs = pd.read_html(StringIO(r.text), encoding='big-5')

    df = pd.concat([df for df in dfs if df.shape[1] <= 11 and df.shape[1] > 5])
    
    if 'levels' in dir(df.columns):
        df.columns = df.columns.get_level_values(1)
    else:
        df = df[list(range(0,10))]
        column_index = df.index[(df[0] == '公司代號')][0]
        df.columns = df.iloc[column_index]
    
    df['當月營收'] = pd.to_numeric(df['當月營收'], 'coerce')
    df = df[~df['當月營收'].isnull()]
    df = df[df['公司代號'] != '合計']
    
    # 偽停頓
    time.sleep(5)

    return df

In [2]:
year=111
month=6
res=monthly_report(year, month)

# 前100個的YoY min為51.3 

In [3]:
yoyrate=res.copy()
minYoY=yoyrate['去年同月增減(%)'].nlargest(100).min()
print('min YoY',minYoY)
yoyfilter=yoyrate['去年同月增減(%)']>minYoY
yoyrate=yoyrate[yoyfilter]
yoyrate.sort_values(['去年同月增減(%)'],ascending=False)

min YoY 57.82


,公司代號,公司名稱,備註,上月比較增減(%),上月營收,去年同月增減(%),去年當月營收,當月營收,前期比較增減(%),去年累計營收,當月累計營收
2,1438,三地開發,"111年累計銷售商品及土地455,714仟元,110年累計銷售商品及土地428仟元,故營收變化大。",11392275.00,4,7594816.66,6,455695,106375.23,428,455714
49,9906,欣巴巴,本月營收與上月之差異，係有個案已全數交屋過戶。累計營收較去年成長之差異，係有新個案銷售入帳所致。,-57.83,737097,2390423.07,13,310768,5262.80,40484,2171079
14,8462,柏文,主係因去年同期因應疫情警戒提升至第三級，配合政府防疫措施，全國廠館暫停營業所致。,0.41,270422,42329.37,640,271548,25.13,1390397,1739881
37,3056,總太,本月營收增加係因個案共好Melody完工交屋。,57513.73,3603,27253.05,7589,2075823,-41.49,3593209,2102110
47,5534,長虹,本月較去年同期增加，係因「雲端科技」、「交響苑」個案增加完成產權移轉予客戶認列收入所致。,171.26,266282,4033.98,17473,722332,418.83,983803,5104290
...,...,...,...,...,...,...,...,...,...,...,...
60,8104,錸寶,110年7月12日取得來穎股權，重編合併報表，更新相關營收數。111/6營收較去年同期大幅增...,12.91,272678,58.82,193862,307892,7.09,1066376,1142088
36,3494,誠研,隨疫情漸趨緩，訂單增加，致使營收成長。,3.25,50761,58.58,33052,52415,29.21,208714,269696
28,5388,中磊,營收增加主因電信寬頻產品出貨增加,24.35,4600905,58.57,3608003,5721230,42.23,19602033,27880916
27,3004,豐達科,主要係客戶需求增加所致,-7.41,194937,58.34,113975,180475,43.38,642898,921798


# 前100個的MoM的min為30.37

In [35]:
momrate=res.copy()
minMoM=momrate['上月比較增減(%)'].nlargest(100).min()
print('min MoM',minMoM)
momdi=momrate['上月比較增減(%)']>minMoM
momrate=momrate[momdi]
momrate=momrate.sort_values(['上月比較增減(%)'],ascending=False)
momrate['公司代號']

min MoM 30.37


1     1438
30    3056
20    4142
16    2527
41    9946
      ... 
2     2704
25    4555
6     2356
82    8249
0     1603
Name: 公司代號, Length: 99, dtype: object

#  分析股價與營收之關係(MoM)

In [4]:
import pandas_datareader as pdr
import datetime as datetime


#  取得前100筆MoM股票代號

In [66]:
t=momrate.set_index('公司代號')
a=[]
for i in range(99):
    a.append(t.index[i]+'.TW')
a[0]

'1438.TW'

In [69]:
start = datetime.datetime(2022,6,30) #from datatime
df = pdr.DataReader(a[0], 'yahoo', start=start)
df['Close']

Date
2022-06-30    31.400000
2022-07-01    30.700001
2022-07-04    31.400000
2022-07-05    32.000000
2022-07-06    32.000000
2022-07-07    32.000000
2022-07-08    32.049999
2022-07-11    33.099998
Name: Close, dtype: float64

In [71]:
df=[]
for i in range(99):
    df.append(pdr.DataReader(a[i], 'yahoo', start=start))

In [73]:
# label data
for i in range(99):
    df[i]['id']=a[i]
    df[i]['rate']=0.0 #新增欄位要先初始化且value決定type

In [100]:
for i in range(99):
    
    if df[i]['rate'].mean>=0:
        with pd.ExcelWriter(f'good.xlsx', engine='xlsxwriter') as writer:
            df[i].to_excel(writer, sheet_name=df[i]['id'])
    else:
        with pd.ExcelWriter(f'bad.xlsx', engine='xlsxwriter') as writer:
            df[i].to_excel(writer, sheet_name=df[i]['id'])
    for j in range(7):
        try:
            value=((df[i]['Close'][j+1]-df[i]['Close'][j])/df[i]['Close'][j])*100
            if j==0:
                df[i]['rate'][j]=0
            df[i]['rate'][j+1]=value
        except IndexError:
#             print(df[i]['id'][j],df[i]['Close'][j])
            if j==0:
                df[i]['rate'][j]=0
        

D:\anaconda\envs\py36\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
D:\anaconda\envs\py36\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
D:\anaconda\envs\py36\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


# 漲幅平均有上漲的有51隻 下降的有49隻

In [119]:
goodc=0
badc=0
with pd.ExcelWriter(f'goodMoM100.xlsx', engine='xlsxwriter') as writer:
    for i in range(99):
        if df[i]['rate'].mean()>=0:
                goodc=goodc+1
                df[i].to_excel(writer, sheet_name=df[i]['id'][0])
with pd.ExcelWriter(f'badMoM100.xlsx', engine='xlsxwriter') as writer2:
    for i in range(99):
        if df[i]['rate'].mean()<0:
                badc=badc+1
                df[i].to_excel(writer2, sheet_name=df[i]['id'][0])
counter=0
with pd.ExcelWriter(f'goodMoM100.xlsx', engine='xlsxwriter') as writer:
    for i in range(99):
        if df[i]['rate'].mean()>=0:
            df[i].to_excel(writer, sheet_name=df[i]['id'][0], index=True, na_rep='NaN')
            # Auto-adjust columns' width
            for column in df[i]:
                column_width = 20
                col_idx = df[i].columns.get_loc(column)
                writer.sheets[df[i]['id'][0]].set_column(col_idx, col_idx, column_width)
with pd.ExcelWriter(f'badMoM100.xlsx', engine='xlsxwriter') as writer2:
    for i in range(99):
        if df[i]['rate'].mean()<0:
            df[i].to_excel(writer2, sheet_name=df[i]['id'][0], index=True, na_rep='NaN')
            # Auto-adjust columns' width
            for column in df[i]:
                column_width = 20
                col_idx = df[i].columns.get_loc(column)
                writer2.sheets[df[i]['id'][0]].set_column(col_idx, col_idx, column_width)

In [121]:
# 'good',goodc,'bad',badc
for i in range(99):
    if df[i]['rate'].mean()>=3:
        print(df[i]['id'])

Date
2022-06-30    6796.TW
2022-07-01    6796.TW
2022-07-04    6796.TW
2022-07-05    6796.TW
2022-07-06    6796.TW
2022-07-07    6796.TW
2022-07-08    6796.TW
2022-07-11    6796.TW
Name: id, dtype: object
Date
2022-06-30    2419.TW
2022-07-01    2419.TW
2022-07-04    2419.TW
2022-07-05    2419.TW
2022-07-06    2419.TW
2022-07-07    2419.TW
2022-07-08    2419.TW
2022-07-11    2419.TW
Name: id, dtype: object


# 平均漲幅最高的為6796 2419

In [127]:
highcodi=momrate['公司代號']=='6796'
momrate[highcodi]

,公司代號,公司名稱,備註,上月比較增減(%),上月營收,去年同月增減(%),去年當月營收,當月營收,前期比較增減(%),去年累計營收,當月累計營收
25,6796,晉弘,-,60.68,28645,14.32,40260,46029,15.01,198910,228768


In [126]:
highcodi=momrate['公司代號']=='2419'
momrate[highcodi]

,公司代號,公司名稱,備註,上月比較增減(%),上月營收,去年同月增減(%),去年當月營收,當月營收,前期比較增減(%),去年累計營收,當月累計營收
4,2419,仲琦,配合客戶需求出貨量增加,54.86,887599,96.2,700587,1374598,6.29,5155444,5480133


# 單日漲幅較高的(>6%)有24隻

In [138]:
a=[]
for i in range(99):
    for j in range(8):
        try:
            if df[i]['rate'][j]>=6:
                print(df[i]['id'][j])
                a.append(i)
                break
        except IndexError:
            break
with pd.ExcelWriter(f'singlehighMoM100.xlsx', engine='xlsxwriter') as writer:
    for i in a:
        df[i].to_excel(writer, sheet_name=df[i]['id'][0])
with pd.ExcelWriter(f'singlehighMoM100.xlsx', engine='xlsxwriter') as writer:
    for i in a:
        df[i].to_excel(writer, sheet_name=df[i]['id'][0], index=True, na_rep='NaN')
            # Auto-adjust columns' width
        for column in df[i]:
            column_width = 20
            col_idx = df[i].columns.get_loc(column)
            writer.sheets[df[i]['id'][0]].set_column(col_idx, col_idx, column_width)

4142.TW
1529.TW
8478.TW
3229.TW
6226.TW
6515.TW
6289.TW
3062.TW
3380.TW
2498.TW
6796.TW
4770.TW
3535.TW
2419.TW
3046.TW
1708.TW
1533.TW
5222.TW
2457.TW
3406.TW
5258.TW
1435.TW
2493.TW
3094.TW


In [139]:
import openpyxl

# 取得excel data(多個sheet)

In [146]:
wb = openpyxl.load_workbook("goodMoM100.xlsx")
test=[]
file_name="goodMoM100.xlsx"
#give the full path of the file here
size=wb.sheetnames
for i in size:
    test.append(pd.read_excel(file_name, sheet_name=i,engine='openpyxl'))
test[1]

,Date,High,Low,Open,Close,Volume,Adj Close,id,rate
0,2022-06-30,41.950001,38.650002,41.750000,39.099998,8788220,39.099998,4142.TW,0.000000
1,2022-07-01,38.900002,36.299999,38.299999,36.400002,5327002,36.400002,4142.TW,-6.905363
2,2022-07-04,37.400002,35.650002,36.400002,36.000000,2349335,36.000000,4142.TW,-1.098905
3,2022-07-05,38.049999,36.400002,36.599998,38.049999,4146326,38.049999,4142.TW,5.694442
4,2022-07-06,41.700001,39.900002,41.650002,40.349998,14158450,40.349998,4142.TW,6.044676
5,2022-07-07,42.500000,39.000000,41.799999,41.500000,15687665,41.500000,4142.TW,2.850066
6,2022-07-08,42.500000,40.400002,41.700001,40.700001,7614622,40.700001,4142.TW,-1.927709
7,2022-07-11,40.700001,39.650002,40.700001,40.349998,4099870,40.349998,4142.TW,-0.859956


# 分析股價與營收之關係(yoy)

In [20]:
yoyrate=res.copy()
minYoY=yoyrate['去年同月增減(%)'].nlargest(100).min()
print('min YoY',minYoY)
yoyfilter=yoyrate['去年同月增減(%)']>minYoY
yoyrate=yoyrate[yoyfilter]
yoyrate=yoyrate.sort_values(['去年同月增減(%)'],ascending=False)
tempYoY=yoyrate.set_index('公司代號')

aYoY=[]
for i in range(99):
    aYoY.append(tempYoY.index[i]+'.TW')
start = datetime.datetime(2022,6,30) #from datatime
df=[]
for i in range(99):
    df.append(pdr.DataReader(aYoY[i], 'yahoo', start=start))


min YoY 57.82


NameError: name 'a' is not defined

In [38]:
for i in range(99):
    df[i].drop(columns=['rate'])

In [39]:
# label data
for i in range(99):
    df[i]['id']=aYoY[i]
    df[i]['rate']=0.0 #新增欄位要先初始化
df[0]

,High,Low,Open,Close,Volume,Adj Close,id,rate
Date,,,,,,,,
2022-06-30,31.799999,31.350000,31.799999,31.400000,19822,31.400000,1438.TW,0.0
2022-07-01,31.150000,30.549999,31.049999,30.700001,39000,30.700001,1438.TW,0.0
2022-07-04,31.750000,30.799999,31.350000,31.400000,80000,31.400000,1438.TW,0.0
2022-07-05,32.099998,31.799999,31.799999,32.000000,61000,32.000000,1438.TW,0.0
2022-07-06,32.049999,31.700001,31.700001,32.000000,135029,32.000000,1438.TW,0.0
2022-07-07,32.150002,31.700001,32.000000,32.000000,115000,32.000000,1438.TW,0.0
2022-07-08,32.099998,32.000000,32.000000,32.049999,33000,32.049999,1438.TW,0.0
2022-07-11,33.349998,32.349998,33.000000,33.099998,570749,33.099998,1438.TW,0.0
2022-07-12,33.150002,32.000000,33.099998,32.250000,88100,32.250000,1438.TW,0.0


In [ ]:
for i in range(99):
    for j in range(7): #新增漲幅
        try:#可能有資料不齊的情況
            value=((df[i]['Close'][j+1]-df[i]['Close'][j])/float(df[i]['Close'][j]))*100
            if j==0:#第一筆沒有漲幅可以算
                df[i]['rate'][j]=0
            df[i]['rate'][j+1]=value
        except IndexError:
            if j==0:
                df[i]['rate'][j]=0
df[0]

In [41]:
df[0].info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 9 entries, 2022-06-30 to 2022-07-12
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   High       9 non-null      float64
 1   Low        9 non-null      float64
 2   Open       9 non-null      float64
 3   Close      9 non-null      float64
 4   Volume     9 non-null      int64  
 5   Adj Close  9 non-null      float64
 6   id         9 non-null      object 
 7   rate       9 non-null      float64
dtypes: float64(6), int64(1), object(1)
memory usage: 648.0+ bytes


In [47]:
def saveYoYDataToExcel(goodName,badName,yoy):
    goodc=0
    badc=0
    df=yoy.copy()
    
#   把好的存入goodName.xls           
    with pd.ExcelWriter(goodName, engine='xlsxwriter') as writer:
        for i in range(99):
            if df[i]['rate'].mean()>=0:
                goodc=goodc+1
                df[i].to_excel(writer, sheet_name=df[i]['id'][0])
#   把壞的存入badName.xls              
    with pd.ExcelWriter(badName, engine='xlsxwriter') as writer2:
        for i in range(99):
            if df[i]['rate'].mean()<0:
                badc=badc+1
                df[i].to_excel(writer2, sheet_name=df[i]['id'][0])
#   修改excel width(goodName)
    with pd.ExcelWriter(goodName, engine='xlsxwriter') as writer:
        for i in range(99):
            if df[i]['rate'].mean()>=0:
                df[i].to_excel(writer, sheet_name=df[i]['id'][0], index=True, na_rep='NaN')
                # Auto-adjust columns' width
                for column in df[i]:
                    column_width = 20
                    col_idx = df[i].columns.get_loc(column)
                    writer.sheets[df[i]['id'][0]].set_column(col_idx, col_idx, column_width)
#   修改excel width(badName)
    with pd.ExcelWriter(badName, engine='xlsxwriter') as writer2:
        for i in range(99):
            if df[i]['rate'].mean()<0:
                df[i].to_excel(writer2, sheet_name=df[i]['id'][0], index=True, na_rep='NaN')
                # Auto-adjust columns' width
                for column in df[i]:
                    column_width = 20
                    col_idx = df[i].columns.get_loc(column)
                    writer2.sheets[df[i]['id'][0]].set_column(col_idx, col_idx, column_width)
    print(f'good:{goodc}, bad:{badc}')
    return df

In [49]:
res=saveYoYDataToExcel('202206goodYoY100.xlsx','202206badYoY100.xlsx',df)
# 'good',goodc,'bad',badc
for i in range(99):
    if res[i]['rate'].mean()>=2:
        print(res[i]['id'][0])

good:46, bad:53
2419.TW
3494.TW


#  又偷跑 = = 內線啦 

In [50]:
con=yoyrate['公司代號']=='3494'
yoyrate[con]

,公司代號,公司名稱,備註,上月比較增減(%),上月營收,去年同月增減(%),去年當月營收,當月營收,前期比較增減(%),去年累計營收,當月累計營收
36,3494,誠研,隨疫情漸趨緩，訂單增加，致使營收成長。,3.25,50761,58.58,33052,52415,29.21,208714,269696


In [59]:
for i in range(99):
    if df[i]['id'][0]=='3494.TW':
        print(i)
df[95]

95


,High,Low,Open,Close,Volume,Adj Close,id,rate
Date,,,,,,,,
2022-06-30,8.02,7.20,8.01,7.45,2807003,7.45,3494.TW,0.000000
2022-07-01,7.54,7.00,7.40,7.00,938092,7.00,3494.TW,-6.040266
2022-07-04,7.40,7.00,7.00,7.30,777070,7.30,3494.TW,4.285717
2022-07-05,8.03,7.43,7.43,8.03,2167438,8.03,3494.TW,9.999993
2022-07-06,8.83,8.04,8.04,8.83,6661130,8.83,3494.TW,9.962643
2022-07-07,9.70,8.51,9.27,9.68,9289161,9.68,3494.TW,9.626278
2022-07-08,9.64,9.00,9.57,9.05,6265814,9.05,3494.TW,-6.508265
2022-07-11,9.42,8.55,9.12,8.95,3669751,8.95,3494.TW,-1.104977
2022-07-12,8.85,8.50,8.68,8.57,1918018,8.57,3494.TW,0.000000


# 單日漲幅大於6%的有26隻 8%有17


In [60]:
a=[]
for i in range(99):
    for j in range(8):
        try:
            if df[i]['rate'][j]>=6:
                print(df[i]['id'][j])
                a.append(i)
                break
        except IndexError:
            break
with pd.ExcelWriter(f'202206singlehighYoY100.xlsx', engine='xlsxwriter') as writer:
    for i in a:
        df[i].to_excel(writer, sheet_name=df[i]['id'][0])
with pd.ExcelWriter(f'202206singlehighYoY100.xlsx', engine='xlsxwriter') as writer:
    for i in a:
        df[i].to_excel(writer, sheet_name=df[i]['id'][0], index=True, na_rep='NaN')
            # Auto-adjust columns' width
        for column in df[i]:
            column_width = 20
            col_idx = df[i].columns.get_loc(column)
            writer.sheets[df[i]['id'][0]].set_column(col_idx, col_idx, column_width)
 

1475.TW
4142.TW
3229.TW
6515.TW
6226.TW
3046.TW
4426.TW
6715.TW
2419.TW
3035.TW
3062.TW
4770.TW
6698.TW
2359.TW
3535.TW
1708.TW
2493.TW
3059.TW
5258.TW
2367.TW
6133.TW
5222.TW
6285.TW
2603.TW
3494.TW
5388.TW


(26,
 [13,
  16,
  25,
  32,
  34,
  35,
  37,
  45,
  46,
  47,
  51,
  56,
  60,
  61,
  62,
  64,
  73,
  74,
  76,
  78,
  81,
  88,
  91,
  93,
  95,
  96])

In [62]:
len(a)

26

In [65]:
len(df[0].index)

9

In [71]:
a=[]
for i in range(99):
    for j in range(8):
        try:
            if df[i]['rate'][j]>=8:
                print(df[i]['id'][j])
                a.append(df[i]['id'][j])
                break
        except IndexError:
            break
len(a)

1475.TW
3229.TW
6226.TW
3046.TW
2419.TW
3035.TW
3062.TW
4770.TW
2359.TW
3535.TW
1708.TW
2493.TW
5258.TW
2367.TW
5222.TW
3494.TW
5388.TW


17

In [74]:
for i in range(len(a)):
    a[i]=a[i].replace('.TW','')
a

['1475',
 '3229',
 '6226',
 '3046',
 '2419',
 '3035',
 '3062',
 '4770',
 '2359',
 '3535',
 '1708',
 '2493',
 '5258',
 '2367',
 '5222',
 '3494',
 '5388']

In [79]:
condi=yoyrate['公司代號']==a[0]
temp=yoyrate[condi]
for i in range(len(a)-1):
    condi=yoyrate['公司代號']==a[i+1]
    temp=pd.concat([temp,yoyrate[condi]], join='inner')
temp.sort_values(['去年同月增減(%)'],ascending=False)

,公司代號,公司名稱,備註,上月比較增減(%),上月營收,去年同月增減(%),去年當月營收,當月營收,前期比較增減(%),去年累計營收,當月累計營收
35,1475,業旺,主要開始銷售染劑產品,-1.08,102710,908.75,10071,101592,588.87,82034,565108
47,3229,晟鈦,海外子公司之營收係以當年度平均匯率換算之，本月份起新增合併個體,137.83,50854,225.05,37209,120948,48.18,216998,321551
52,6226,光鼎,本月認列房產收入,125.07,71184,119.62,72949,160217,23.60,445156,550213
31,3046,建��,111年6月份及1-6月累計營收較去年同期增加，主係新增大型專案業務,53.70,296611,117.78,209340,455907,50.69,1059612,1596799
5,2419,仲琦,配合客戶需求出貨量增加,54.86,887599,96.20,700587,1374598,6.29,5155444,5480133
28,3035,智原,因量產營收增加,-0.83,1126473,96.03,569812,1117013,103.90,3222827,6571587
16,3062,建漢,因疫情趨緩及市場缺料情況也逐漸趨緩，故本月營收金額較去年同期增加。,76.66,352742,92.99,322901,623169,-2.46,2004226,1954919
27,4770,上品,較去年同期增加，主係本期對銷貨客戶之設備出貨增加所致,56.55,410895,90.04,338481,643267,79.35,1659967,2977217
3,2359,所羅門,係智動化事業群及發電機事業群之客戶擴廠致需求增加，使得本月較去年同期增加50%以上。,3.20,500897,87.15,276230,516975,51.90,1910606,2902342
30,3535,晶彩科,本公司以銷售機台為主，111年6月出售機台較去年同期增加，此為產業特性，尚屬正常。,55.26,99446,87.14,82503,154402,13.71,732620,833123


# 查價格

In [108]:
for i in range(99):
    if df[i]['id'][0]=='2367.TW':
        j=i
df[j]

,High,Low,Open,Close,Volume,Adj Close,id,rate
Date,,,,,,,,
2022-06-30,17.650000,16.750000,17.650000,16.75,3945980,16.75,2367.TW,0.000000
2022-07-01,16.700001,15.550000,16.400000,15.55,4205087,15.55,2367.TW,-7.164178
2022-07-04,16.049999,15.450000,15.750000,15.80,2661522,15.80,2367.TW,1.607717
2022-07-05,16.350000,15.800000,16.100000,16.10,2156080,16.10,2367.TW,1.898735
2022-07-06,16.200001,15.100000,16.200001,15.10,3207365,15.10,2367.TW,-6.211180
2022-07-07,16.600000,15.700000,15.900000,16.60,6542001,16.60,2367.TW,9.933775
2022-07-08,17.250000,16.500000,16.700001,16.85,7061628,16.85,2367.TW,1.506024
2022-07-11,16.950001,16.200001,16.750000,16.75,2864628,16.75,2367.TW,-0.593474
2022-07-12,16.600000,16.200001,16.600000,16.25,1464976,16.25,2367.TW,0.000000


### MOM&YOY同時上升
- 3229趨勢向上(營收一公布後噴漲停且在禮拜二跌近400點逆勢漲)           
  - MoM 137.83 YoY 225.05
- 6226持平(但營收公布前一天噴漲停) 後來向下                          
  - MoM 125.07 YoY 119.62
- 3046趨勢向上(營收公布前三天連噴漲停公布後開始下修)                  
  - MoM 53.70  YoY 117.78
- 4770趨勢向下(但營收公布前一天噴漲停)且公布後指跌                    
  - MoM 56.55  YoY 90.04
- 5258持平(營收一公布後噴漲停且在禮拜二跌近400點逆勢漲)               
  - MoM  41.54  YoY  68.88
- 5222趨勢向下(營收公布前接近漲停，一公布後失去熱度量大幅下降並且小跌)  
  - MoM 45.13 YoY 61.74
- 3062持平公布前兩天噴8.7%，但是在更早之前跌9% 營收公布後漲6.2%
#### 營收一公布後噴漲停的股票大多是沒偷跑的
#### 營收公布前就噴的大多在公布後修正

### MoM沒上升&YoY大幅上升
- 1475趨勢向上營收公布前噴一天漲停，公布後連續小漲
  - MoM -1.08 YoY 908.75
- 3035趨勢向下一直下修營收公布前有一天7/5漲8.5% 其他天都大跌
  - MoM -0.83 YoY 96.03
 
### MoM持平 & YoY 大幅上升
- 2359趨勢向上營收公布前接近漲停，公布後續漲4%
  - MoM 3.20 YoY 87.15
- 3494趨勢向上(營收公布前三天連噴漲停公布後開始下修)
  - MoM 3.25 YoY 58.58
- 2367公布前噴漲停一公布量縮剩1/3且開始小跌
  - MoM 0.2  YoY 67.30

IndexError: index 2 is out of bounds for axis 0 with size 2